In [ ]:
## 🤖 4. Machine Learning (4_machine_learning.ipynb)

# Importar paquetes
import pandas as pd
import numpy as np

# Model Selection y Métricas
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

# Pipelines (Aunque aquí no se usa el objeto Pipeline, sí se usa la lógica de entrenamiento)
from sklearn.pipeline import Pipeline 

# Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB

# Para guardar el modelo
import pickle

# --- 1. Carga de datos procesados ---
df = pd.read_csv('./data/titanic_procesado.csv')

# --- 2. División de datos (X y Y) ---
X = df.drop(['Survived'], axis=1) # Características
y = df['Survived']               # Variable Objetivo

# --- 3. División Entrenamiento-Prueba (80/20) ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convertir a NumPy arrays (como se sugirió en el proceso)
X_train = X_train.values
y_train = y_train.values
X_test = X_test.values
y_test = y_test.values

print(f"Dimensiones de entrenamiento: X_train={X_train.shape}, y_train={y_train.shape}")
print(f"Dimensiones de prueba: X_test={X_test.shape}, y_test={y_test.shape}")

# --- 4. Definición de Modelos e Hiperparámetros para GridSearch ---
modelos = {
    'Regresión Logística': {
        'modelo': LogisticRegression(solver='liblinear'),
        'parametros': {
            'C': [0.01, 0.1, 1, 10, 100],
            'penalty': ['l1', 'l2']
        }
    },
    'Clasificador de Vectores de Soporte': {
        'modelo': SVC(random_state=42),
        'parametros': {
            'kernel': ['linear', 'rbf'], # Reducido para velocidad
            'C': [0.1, 1, 10]
        }
    },
    'Clasificador de Árbol de Decisión': {
        'modelo': DecisionTreeClassifier(random_state=42),
        'parametros': {
            'splitter': ['best', 'random'],
            'max_depth': [None, 3, 5] # Reducido
        }
    },
    'Clasificador de Bosques Aleatorios': {
        'modelo': RandomForestClassifier(random_state=42),
        'parametros': {
            'n_estimators': [10, 100],
            'max_depth': [None, 3, 5],
            'max_features': ['sqrt', 'log2']
        }
    },
    'Clasificador de Gradient Boosting': {
        'modelo': GradientBoostingClassifier(random_state=42),
        'parametros': {
            'n_estimators': [10, 100],
            'max_depth': [1, 3] # Reducido
        }
    },
    'Clasificador AdaBoost': {
        'modelo': AdaBoostClassifier(random_state=42),
        'parametros': {
            'n_estimators': [10, 100]
        }
    },
    'Clasificador K-Nearest Neighbors': {
        'modelo': KNeighborsClassifier(),
        'parametros': {
            'n_neighbors': [3, 5, 7] # Hiperparámetro clave
        }
    },
    'Clasificador XGBoost': {
        'modelo': XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
        'parametros': {
            'n_estimators': [10, 100],
            'max_depth': [1, 3]
        }
    },
    'Clasificador LGBM': {
        'modelo': LGBMClassifier(random_state=42, verbose=-1),
        'parametros': {
            'n_estimators': [10, 100],
            'max_depth': [1, 3],
        }
    },
    'GaussianNB': {
        'modelo': GaussianNB(),
        'parametros': {}
    },
    'Clasificador Naive Bayes': {
        'modelo': BernoulliNB(),
        'parametros': {
            'alpha': [0.1, 1.0] # Reducido
        }
    }
}

# --- 5. Entrenamiento y Selección del Mejor Modelo (GridSearch) ---
puntajes_modelos = []
mejor_precision = 0
mejor_estimador = None
mejor_modelo = None

for nombre, info_modelo in modelos.items():
    print(f"Entrenando {nombre}...")
    
    # 5.1. Inicializar GridSearchCV
    grid_search = GridSearchCV(
        estimator=info_modelo['modelo'],
        param_grid=info_modelo['parametros'],
        cv=5,
        scoring='accuracy',
        verbose=0,
        n_jobs=-1,
    )

    # 5.2. Ajustar GridSearchCV con los datos de entrenamiento
    grid_search.fit(X_train, y_train)
    
    # 5.3. Hacer predicciones
    y_pred = grid_search.predict(X_test)
    
    # 5.4. Calcular la precisión
    precision = accuracy_score(y_test, y_pred)
    
    # 5.5. Almacenar resultados
    puntajes_modelos.append({
        'Modelo': nombre,
        'Precisión': precision
    })
    
    # 5.6. Actualizar el mejor modelo
    if precision >= mejor_precision: # Usamos >= para incluir empates si se desea
        mejor_modelo = nombre
        mejor_precision = precision
        mejor_estimador = grid_search.best_estimator_
        # Imprimir resultados del mejor modelo actual para seguimiento
        print(f"Nuevo Mejor Modelo: {mejor_modelo} con Precisión: {mejor_precision:.4f}")

# --- 6. Mostrar Resultados Finales ---
metricas = pd.DataFrame(puntajes_modelos).sort_values('Precisión', ascending=False)
print("\n" + "="*50)
print("Rendimiento de los modelos de clasificación")
print(metricas.round(4))
print("="*50)
print("MEJOR MODELO DE CLASIFICACIÓN (Resultado del GridSearch)")
print(f"Modelo: {mejor_modelo}")
print(f"Precisión: {mejor_precision:.4f}")

# --- 7. Guardar el Mejor Modelo ---
# NOTA: En este notebook guardamos el modelo de la versión "manual" (sin Pipeline)
with open('modelo_ml_entrenado.pkl', 'wb') as archivo_estimador:
    pickle.dump(mejor_estimador, archivo_estimador)
print(f"\nEl mejor estimador ({mejor_modelo}) guardado como: modelo_ml_entrenado.pkl")